In [1]:
import pandas as pd
import time
import re
import pathlib
import os
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
distrito = int(input('Indique el número del distrito en el que desea revisar la votación (1-28): '))
df = pd.read_csv(f'Distrito{distrito}\Dist{distrito}_Pleb2020.csv')

Introducir el número del distrito en el que desea revisar la votación (1-28)
2


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [4]:
driver_path = os.path.join(pathlib.Path().absolute(), 'chrome_driver\\chromedriver.exe')
driver = webdriver.Chrome(executable_path = driver_path, options=options)
driver.get('https://pv.servelelecciones.cl/')

In [5]:
%%time

#PESTAÑA SUPERIOR: "Participación"
step1 = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[1]/a'
WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step1))).click()

#PANEL LATERAL IZQUIERDO: "En Chile"
step2 = '/html/body/div[2]/div[1]/div[2]/div/ul/li[3]/a'
WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step2))).click()

#DISTRITO
step3 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[6]/option[{distrito + 1}]'
WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step3))).click()


def Comuna(df):
    return df['COMUNA'].unique()

def Circuns(df, i):
    return df.loc[(df['COMUNA']==Comuna(df)[i])]['CIRCUNSCRIPCION'].unique()

def Colegio(df, i, j):
    return df.loc[(df['COMUNA']==Comuna(df)[i]) & (df['CIRCUNSCRIPCION']==Circuns(df, i)[j])]['COLEGIO'].unique()

def Mesa(df, i, j, k):
    return df.loc[(df['COMUNA']==Comuna(df)[i]) & (df['CIRCUNSCRIPCION']==Circuns(df, i)[j]) & (df['COLEGIO']==Colegio(df, i, j)[k])]['MESA'].unique()

#COMUNA
for m in range(len(Comuna(df))):
    time.sleep(3)
    step4 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[9]/option[{m+2}]'
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step4))).click()
    #CIRCUNSCRIPCION ELECTORAL
    for j in range(len(Circuns(df, m))):
        time.sleep(3)
        step5 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[10]/option[{j+2}]'
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step5))).click()
        #COLEGIO
        for k in range(len(Colegio(df, m, j))):
            #time.sleep(3)
            step6 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[11]/option[{k+2}]'
            WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step6))).click()
            time.sleep(4)
            #MESA
            for q in range(len(Mesa(df, m, j, k))):
                step7 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[12]/option[{q+2}]'
                WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, step7))).click()        
                time.sleep(0.3)
                df3 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
                df3 = df3[df3 != '\n'].dropna(axis=0).reset_index(drop=True)
                df3[0] = df3[0].apply(lambda x: "".join(re.split("( )", x)))
                df3 = df3[0].str.split(expand=True)
                df.loc[(df['MESA'] == Mesa(df, m, j, k)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, m)[j]), 'APRUEBO'] = df3[1][0]
                df.loc[(df['MESA'] == Mesa(df, m, j, k)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, m)[j]), 'RECHAZO'] = df3[1][1]

Wall time: 10min 53s


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MESA             865 non-null    object 
 1   TOTALMESAS       865 non-null    int64  
 2   ELECTORES        865 non-null    int64  
 3   EMITIDOS         865 non-null    int64  
 4   PORCENTAJE       865 non-null    float64
 5   COLEGIO          865 non-null    object 
 6   COMUNA           865 non-null    object 
 7   CIRCUNSCRIPCION  865 non-null    object 
 8   APRUEBO          865 non-null    object 
 9   RECHAZO          865 non-null    object 
dtypes: float64(1), int64(3), object(6)
memory usage: 67.7+ KB


In [7]:
df['APRUEBO'] = df['APRUEBO'].astype(int)
df['RECHAZO'] = df['RECHAZO'].astype(int)

In [8]:
df.to_csv(f'Distrito{distrito}\Dist{distrito}_Pleb2020_Votacion.csv', index=False)